In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from matplotlib import pyplot as plt

%matplotlib inline

In [39]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

word_index = keras.datasets.imdb.get_word_index()

In [48]:
top_values = [i for i in word_index if word_index[i]<10]
print(top_values)

['it', 'is', 'in', 'of', 'a', 'br', 'the', 'and', 'to']


In [68]:
def get_word(word_index, idx):
    l = [word for word in word_index if word_index[word] == idx]
    return l[0]

#Como las reviews ya vienen ordenadas por index, no puedo imprimir el texto,
#Solo una secuenca de palabras ordenadas por frecuencia de aparicion
def print_review(review, word_index):
    streview = []
    for idx in review:
        streview.append(get_word(word_index, idx))
    print(' '.join(streview))
    return None

print_review(x_train[9], word_index)

the as on there plot she's iii film that for find that saw better just is along wrong silly awesome or play this you doing was one in own that successful are make and old plot gets unfortunately of on was although except value omar that with her do they gets for that with timing really way that is played character i i what poor set but is along 100 studio on film is missing br received fact to is mercifully br fabulous and them powers is tapes br enjoys indicate good women show to one good played i i was plain film because avoid for of totally it time do period it couple in college in viewers get br of my to of material it yet br out more


In [22]:
def preprocessing(X, word_index):


1641221/1641221 [==============================] - 0s 0us/step


In [38]:
len(x_train[10])

450